In [59]:
import numpy as np
import pandas as pd

from itertools import product, combinations

In [60]:
with open('../../inputs/0107_network.txt', 'r') as f:
    network_string = f.read()

This problem, I used a simple backtracking approach.

We sort the edges in order of weight, from greatest to least (since we want to remove high-weight edges first). Then for each edge we remove it from the graph, check if the graph is still connected (quick DFS or BFS). If it's no longer connected, then put the edge back. If it is connected then we remove it and move to the next one.

In [202]:
# dfs to check if graph is connected
def connected(start_node, graph, suppress = True):
    n = len(graph)
    visited = np.zeros(n)
    queue = [start_node]

    while queue:
        if not suppress: print(queue)

        current = queue.pop(0)

        # use np vectorization to quickly find all nodes to append to queue
        # find all connected nodes
        to_add = np.where(graph[current] != 0)[0]
        # append any nodes that have not been visited onto the end of queue
        queue += list(to_add[np.where(visited[to_add] == 0)[0]])

        visited[current] = 1

    if not suppress: print(np.where(visited == 0))
    return np.sum(visited) == len(graph)

In [203]:
network = np.array([[0 if c == '-' else int(c) for c in vertex.split(',')] for vertex in network_string.split('\n')], int)

# # get example network here for testing
# network = np.array([
#     [0, 16, 12, 21, 0, 0, 0],
#     [16, 0, 0, 17, 20, 0, 0],
#     [12, 0, 0, 28, 0, 31, 0],
#     [21, 17, 28, 0, 18, 19, 23],
#     [0, 20, 0, 18, 0, 0, 11],
#     [0, 0, 31, 19, 0, 0, 27],
#     [0, 0, 0, 23, 11, 27, 0]
# ], int)

In [206]:
# we want to remove edges with maximum weights
g = network.copy()
# for i,j in combinations(range(len(g)), 2):
#     g[j,i] = 0

# every edge is doubled
old_sum = np.sum(g) // 2

# order weights least to greatest
weights = sorted(
    {(i,j): g[i,j] for i,j in combinations(range(len(g)), 2)}.items(), 
    reverse = True, 
    key = lambda a: (a[1],a[0])
)

# use backtracking to check if we can remove an edge and still have a connected graph
# we can use dfs or bfs to check if the graph remains connected after removing that edge
for (i,j), w in weights:
    old_g = g.copy()

    # remove both copies of edge
    g[i,j] = 0
    g[j,i] = 0

    # if graph is no longer connected, then that edge is essential
    if not connected(0, g):
        g = old_g.copy()

# every edge is doubled
new_sum = np.sum(g) // 2

print(f'old total weight: {old_sum}\nnew total weight: {new_sum}\nimprovement: {old_sum - new_sum}')

old total weight: 261832
new total weight: 2153
improvement: 259679
